In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from keras.metrics import Recall
from keras.metrics import Accuracy

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
df = pd.read_csv('SQL_Dataset_1.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32996 entries, 0 to 32995
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Query   32996 non-null  object
 1   Label   32996 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 515.7+ KB


In [5]:
X=df['Query']
y=df['Label']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
max_words = 1000
max_len = 200
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [8]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,100,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [9]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=[Recall()])


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 200)]             0         
                                                                 
 embedding (Embedding)       (None, 200, 100)          100000    
                                                                 
 lstm (LSTM)                 (None, 64)                42240     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257  

In [10]:
model.fit(sequences_matrix,y_train,batch_size=128,epochs=20,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/20

165/165 [==============================] - 53s 295ms/step - loss: 0.1645 - recall: 0.8457 - val_loss: 0.0448 - val_recall: 0.9810
Epoch 2/20
165/165 [==============================] - 46s 280ms/step - loss: 0.0408 - recall: 0.9796 - val_loss: 0.0380 - val_recall: 0.9824
Epoch 3/20
165/165 [==============================] - 47s 285ms/step - loss: 0.0350 - recall: 0.9834 - val_loss: 0.0357 - val_recall: 0.9843
Epoch 4/20
165/165 [==============================] - 47s 286ms/step - loss: 0.1082 - recall: 0.9703 - val_loss: 0.1182 - val_recall: 0.9302


In [11]:
X_test_sequences = tok.texts_to_sequences(X_test)
X_test_sequences_matrix = sequence.pad_sequences(X_test_sequences,maxlen=max_len)

In [12]:
accuracy = model.evaluate(X_test_sequences_matrix,y_test)

207/207 [==============================] - 9s 42ms/step - loss: 0.1200 - recall: 0.9269


In [13]:
print('Test set\n Loss: {:0.3f}\n Accuracy: {:0.3f}'.format(accuracy[0],accuracy[1]))

Test set
 Loss: 0.120
 Accuracy: 0.927


In [14]:
y_pred=model.predict(X_test_sequences_matrix)

207/207 [==============================] - 9s 39ms/step


In [15]:
print(y_pred)

[[0.03030549]
 [0.01086133]
 [0.0182684 ]
 ...
 [0.01471641]
 [0.0154811 ]
 [0.93764937]]


In [16]:
for i in range(len(y_pred)):
    if y_pred[i]>0.5:
        y_pred[i]=1
    elif y_pred[i]<=0.5:
        y_pred[i]=0

In [17]:
print(y_pred)

[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [1.]]


In [18]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
confusion_matrix(y_test, y_pred)  
plt.show()
accuracy= accuracy_score(y_test, y_pred)
precision= precision_score(y_test, y_pred, zero_division=1)
recall= recall_score(y_test, y_pred, zero_division=1)
f1_score= f1_score(y_test, y_pred, zero_division=1)
print(" Accuracy : {0} \n Precision : {1} \n Recall : {2} \n F1 : {3}".format(accuracy, precision, recall, f1_score))

 Accuracy : 0.9669696969696969 
 Precision : 0.9907852564102564 
 Recall : 0.926911544227886 
 F1 : 0.9577846630518978


In [21]:
from keras.models import load_model
from pathlib import Path

model.save_weights("lstm_weights.h6")
print('done')

done


In [25]:
loaded_model = tf.keras.models.load_model('lstm_model.h6')
loaded_model.load_weights('lstm_weights.h6')

OSError: No file or directory found at lstm_model.h6